<a href="https://colab.research.google.com/github/solobala/FEML23/blob/master/FEML23_DZ11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание к лекции «Улучшение качества модели» обновленное

**Цель:**

Применить на практике алгоритмы по автоматической оптимизации параметров моделей машинного обучения.

**Описание задания:**

В домашнем задании нужно решить задачу классификации наличия болезни сердца у пациентов наиболее эффективно. Данные для обучения моделей необходимо загрузить самостоятельно с сайта. Целевая переменная – наличие болезни сердца (HeartDisease). Она принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. Подробное описание признаков можно прочесть в описании датасета на сайте. Для выполнения работы не обязательно вникать в медицинские показатели.

# 0. Установка пакетов, импорт библиотек и пользовательские функции

In [1]:
# установка catboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 11.0 MB/s eta 0:00:00


In [2]:
# Установка библиотеки для работы с kaggle
!pip install -q kaggle

In [3]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import  BaggingClassifier
from sklearn.ensemble import  StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

from sklearn.model_selection import KFold
from sklearn.model_selection import  cross_validate
import seaborn as sns
from matplotlib import pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 1.Получите данные и загрузите их в рабочую среду. 

In [4]:
# Это напоминалка - приглашение загрузить в сессионное хранилище свой kaggle.json. Ключ стерла
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"marinabalakina","key":"0763a280a2e52ea690b44499c8ab213e"}'}

In [5]:
! mkdir ~/.kaggle # Создаем каталог .kaggle  1 раз, для того, чтобы сюда положить ключ kaggle.json, который получаем на Kaggle при регистрации

In [6]:
! cp kaggle.json ~/.kaggle/  # кладем ключ

In [7]:
! chmod 600 /root/.kaggle/kaggle.json # засекречиваем. К сожалению, это все только на время сеанса

In [8]:
!kaggle datasets list -s 'heart' # Сюда  - люббое ключевое слово

ref                                                          title                                        size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------  ------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
andrewmvd/heart-failure-clinical-data                        Heart Failure Prediction                      4KB  2020-06-20 01:03:20         110928       1993  1.0              
fedesoriano/heart-failure-prediction                         Heart Failure Prediction Dataset              9KB  2021-09-10 18:11:57         101615       2141  1.0              
johnsmith88/heart-disease-dataset                            Heart Disease Dataset                         6KB  2019-06-06 15:33:55          84262        663  0.88235295       
rashikrahmanpritom/heart-attack-analysis-prediction-dataset  Heart Attack Analysis & Prediction Dataset    4KB  202

In [9]:
!kaggle datasets download -d 'fedesoriano/heart-failure-prediction'  # А это нужный датасет

  0% 0.00/8.56k [00:00<?, ?B/s]
100% 8.56k/8.56k [00:00<00:00, 34.6MB/s]


In [10]:
!unzip heart-failure-prediction.zip

Archive:  heart-failure-prediction.zip
  inflating: heart.csv               


In [11]:
df = pd.read_csv('/content/heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [12]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


#2. Подготовьте датасет к обучению моделей:
a) Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder. 

In [13]:
df.info() # пропусков типа nan нет

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [14]:
# проверим, нет ли пропусков, обозначенных "?" 
flag=True
for column in df.columns:
  if('?')  in list(df[column].value_counts()):
    print(column)
    flag = False
if flag:
    print('пропусков "?" нет')

пропусков "?" нет


In [15]:
# Категориальные features - определяем по типу object. Вместе с target их 9
cat_features = df.select_dtypes("object").columns.tolist()
print(*cat_features, sep=", ")

Sex, ChestPainType, RestingECG, ExerciseAngina, ST_Slope


In [16]:
# Это числовые признаки
num_features = df.select_dtypes(['int', 'float']).columns.tolist()
print(*num_features, sep=", ")

Age, RestingBP, Cholesterol, FastingBS, MaxHR, Oldpeak, HeartDisease


In [17]:
# Настройка параметров оценивания алгоритма
random_seed=0
num_folds = 10
n_iter = 1000
n_estimators = 100
scoring = {"accuracy": make_scorer(accuracy_score), "recall": make_scorer(recall_score), "precision": make_scorer(precision_score), "f1": make_scorer(f1_score) }
# scoring = ['accuracy','recall','precision','f1'] # F1 = 2 * (precision * recall) / (precision + recall) Непонятно, зачем использовать Cross_validate Вместо cross_val_score, если время обучения и предсказания не используем???
X = df.drop(columns=['HeartDisease'])
y=df['HeartDisease']

target_names = ['healthy', 'sick']

In [18]:
# Формирование списка алгоритмов
models = []
models.append(('LR_Base', LogisticRegression(random_state=random_seed)))
models.append(('DTC', DecisionTreeClassifier(random_state=random_seed)))
models.append(('RFC', RandomForestClassifier(random_state=random_seed)))
models.append(('BGC', BaggingClassifier(random_state=random_seed)))
models.append(('SCC', StackingClassifier(
     [
                                      ('DTS', DecisionTreeClassifier(random_state=random_seed)),
                                      ('RFC',  RandomForestClassifier(random_state=random_seed)),
                                     ],  
                                      LinearSVC(C=1, random_state=random_seed) 
)))
models.append(('GBC', GradientBoostingClassifier(random_state=random_seed)))
models.append(('ABC', AdaBoostClassifier(random_state=random_seed)))
models.append(('LGBMC', LGBMClassifier(random_state=random_seed)))
models.append(('XGBC', XGBClassifier(random_state=random_seed)))
models.append(('CBC', CatBoostClassifier(random_state=random_seed)))


In [19]:
# Пайплайн трансформации числовых и категориальных признаков
num_features =['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']
numeric_transformer = Pipeline(
    steps=[ ("scaler", Normalizer())]
)


categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features),
    ]
)  

In [20]:
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression(random_state=random_seed))]
)

In [21]:
# Блиц-проверка алгоритмов на нормализованных исходных данных
# (функция Normalizer)
pipelines = []
pipelines.append(('Base_LR', Pipeline([("preprocessor", preprocessor),
                                    ('LR_Base', LogisticRegression(random_state=random_seed))])))
pipelines.append(('SS_DTS', Pipeline([("preprocessor", preprocessor),
                                    ('DTS', DecisionTreeClassifier(random_state=random_seed))])))

pipelines.append(('SS_RFC', Pipeline([("preprocessor", preprocessor),
                                     ('RFC', RandomForestClassifier(random_state=random_seed))])))

pipelines.append(('SS_BGS', Pipeline([("preprocessor", preprocessor),
                                     ('BGC', BaggingClassifier(random_state=random_seed))])))

pipelines.append(('SS_GBC', Pipeline([("preprocessor", preprocessor),
                                      ('GBC', GradientBoostingClassifier(random_state=random_seed))])))

pipelines.append(('SS_ABC', Pipeline([("preprocessor", preprocessor),
                                      ('ABC', AdaBoostClassifier(random_state=random_seed))])))

pipelines.append(('SS_LGBMC', Pipeline([("preprocessor", preprocessor),
                                      ('LGBMC', LGBMClassifier(random_state=random_seed))])))

pipelines.append(('SS_XGBC', Pipeline([("preprocessor", preprocessor),
                                    ('XGBC', XGBClassifier(random_state=random_seed))])))

pipelines.append(('SS_SCC', Pipeline([("preprocessor", preprocessor),
                                     ('SCC', StackingClassifier([
                                      ('DTS', DecisionTreeClassifier(random_state=random_seed)),
                                      ('RFC',  RandomForestClassifier(random_state=random_seed))],  
                                      LinearSVC(C=1, random_state=random_seed)))])))

pipelines.append(('SS_CBC', Pipeline([("preprocessor", preprocessor),
                                      ('CBC', CatBoostClassifier(random_state=random_seed,
                                         verbose=False
                                      ))])))

#3. Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

In [55]:
model= LogisticRegression(random_state=random_seed)
model.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [56]:
cv_results = cross_validate(model, X, y, cv=num_folds, scoring=
                                 scoring,  return_train_score=True)
res = {'fit_time': cv_results['fit_time'].mean(), 
                       'train_accuracy':  cv_results['train_accuracy'].mean(), 
                      'train_precision':  cv_results['train_precision'].mean(),
                      'train_recall':  cv_results['train_recall'].mean(),
                      'train_f1':  cv_results['train_f1'].mean(),
                      'score_time':   cv_results['score_time'].mean(), 
                      'test_accuracy':  cv_results['test_accuracy'].mean(), 
                      'test_precision':  cv_results['test_precision'].mean(),
                      'test_recall':  cv_results['test_recall'].mean(),
                      'test_f1':  cv_results['test_f1'].mean(),
                      }

In [57]:
res

{'fit_time': 0.022654271125793456,
 'train_accuracy': 0.8731517401500801,
 'train_precision': 0.8728534253891393,
 'train_recall': 0.9022268831280517,
 'train_f1': 0.887289726633596,
 'score_time': 0.00782637596130371,
 'test_accuracy': 0.8483874820831343,
 'test_precision': 0.8644599660790575,
 'test_recall': 0.8675686274509802,
 'test_f1': 0.8615900831638843}

In [23]:
# Оценивание эффективности выполнения каждого алгоритма
results = {}

for name, model in pipelines:
    kfold = KFold(n_splits=num_folds, random_state=random_seed, shuffle=True)
    X = df.drop(columns=['HeartDisease'])
    y=df['HeartDisease']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)
    m_fit = model.fit(X_train, y_train) # обучение
    m_predict = model.predict(X_test) # предсказание
  
    cv_results = cross_validate(model, X, y, cv=num_folds, scoring=
                                 scoring,  return_train_score=True)
   
    results[f'{name}'] = {'fit_time': cv_results['fit_time'].mean(), 
                       'train_accuracy':  cv_results['train_accuracy'].mean(), 
                      'train_precision':  cv_results['train_precision'].mean(),
                      'train_recall':  cv_results['train_recall'].mean(),
                      'train_f1':  cv_results['train_f1'].mean(),
                      'score_time':   cv_results['score_time'].mean(), 
                      'test_accuracy':  cv_results['test_accuracy'].mean(), 
                      'test_precision':  cv_results['test_precision'].mean(),
                      'test_recall':  cv_results['test_recall'].mean(),
                      'test_f1':  cv_results['test_f1'].mean(),
                      }

#4. Обучите модель логистической регрессии с параметрами по умолчанию.

In [24]:
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   Normalizer())]),
                                                  ['Age', 'RestingBP',
                                                   'Cholesterol', 'FastingBS',
                                                   'MaxHR', 'Oldpeak']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('selector',
                                                                   SelectPercentile(percentile=50,
                                                                                    score_func=<function chi2 at 0x7fba29d22320>))]),
                                                  ['Sex', 'ChestPainType',
                                                   'RestingECG',
                                                   'ExerciseAngina',
                                                   'ST_Slope'])])),
                ('classifier', LogisticRegression(random_state=0))])

# 5. Подсчитайте основные метрики модели. Используйте следующие метрики и функцию:
cross_validate(…, cv=10, scoring=[‘accuracy’,‘recall’,‘precision’,‘f1’])

In [25]:
cv_results = cross_validate(clf, X, y, cv=num_folds, scoring=
                                 scoring,  return_train_score=True)

In [26]:
results['clf'] = {'fit_time': cv_results['fit_time'].mean(), 
                       'train_accuracy':  cv_results['train_accuracy'].mean(), 
                      'train_precision':  cv_results['train_precision'].mean(),
                      'train_recall':  cv_results['train_recall'].mean(),
                      'train_f1':  cv_results['train_f1'].mean(),
                      'score_time':   cv_results['score_time'].mean(), 
                      'test_accuracy':  cv_results['test_accuracy'].mean(), 
                      'test_precision':  cv_results['test_precision'].mean(),
                      'test_recall':  cv_results['test_recall'].mean(),
                      'test_f1':  cv_results['test_f1'].mean(),
                      }

In [27]:
results['clf']

{'fit_time': 0.020459294319152832,
 'train_accuracy': 0.869037127691033,
 'train_precision': 0.8667722684696596,
 'train_recall': 0.9020099758248689,
 'train_f1': 0.8840322431646668,
 'score_time': 0.010377860069274903,
 'test_accuracy': 0.8483874820831343,
 'test_precision': 0.8621312033016387,
 'test_recall': 0.8676470588235293,
 'test_f1': 0.8613655536558097}

In [28]:
results['Base_LR']

{'fit_time': 0.01967000961303711,
 'train_accuracy': 0.869037127691033,
 'train_precision': 0.8667722684696596,
 'train_recall': 0.9020099758248689,
 'train_f1': 0.8840322431646668,
 'score_time': 0.01035451889038086,
 'test_accuracy': 0.8483874820831343,
 'test_precision': 0.8621312033016387,
 'test_recall': 0.8676470588235293,
 'test_f1': 0.8613655536558097}

#6. Оптимизируйте 3-4 параметра модели:

## a) Используйте GridSearchCV.

In [52]:
SelectedColumns=df[cat_features]
encoded_features = pd.get_dummies(SelectedColumns, columns=cat_features,drop_first=True)

In [53]:
other_features=df.loc[:, num_features]
X = pd.concat([other_features, encoded_features], axis=1)
transformer = Normalizer().fit(X)
transformer.transform(X)

array([[0.1091459 , 0.38201066, 0.78857914, ..., 0.        , 0.        ,
        0.00272865],
       [0.16832526, 0.54963349, 0.61833768, ..., 0.        , 0.00343521,
        0.        ],
       [0.11260514, 0.39563968, 0.86127716, ..., 0.        , 0.        ,
        0.00304338],
       ...,
       [0.25354517, 0.57826092, 0.58270908, ..., 0.00444816, 0.00444816,
        0.        ],
       [0.17497262, 0.39906036, 0.72444804, ..., 0.        , 0.0030697 ,
        0.        ],
       [0.13348049, 0.48474495, 0.6147128 , ..., 0.        , 0.        ,
        0.00351264]])

In [44]:
import warnings
warnings.filterwarnings('ignore')
# parameter grid
penalty = ["l1", "l2", "elasticnet"]
l1_ratio = np.linspace(0, 1, 20)
C = np.logspace(0, 10, 20)
param_grid = {"penalty" : penalty,
             "l1_ratio" : l1_ratio,
             "C" : C}

In [58]:
LR_grid_model = LogisticRegression(solver='saga', max_iter=5000, class_weight = "balanced")

LR_grid_model = GridSearchCV(LR_grid_model, param_grid = param_grid)
LR_grid_model.fit(X_train, y_train)

GridSearchCV(estimator=LogisticRegression(class_weight='balanced',
                                          max_iter=5000, solver='saga'),
             param_grid={'C': array([1.00000000e+00, 3.35981829e+00, 1.12883789e+01, 3.79269019e+01,
       1.27427499e+02, 4.28133240e+02, 1.43844989e+03, 4.83293024e+03,
       1.62377674e+04, 5.45559478e+04, 1.83298071e+05, 6.15848211e+05,
       2.06913808e+06, 6.95192796e+06, 2.33572147e+07, 7.84759970e+07,
       2.63665090e+08, 8.85866790e+08, 2.97635144e+09, 1.00000000e+10]),
                         'l1_ratio': array([0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
       0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
       0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
       0.78947368, 0.84210526, 0.89473684, 0.94736842, 1.        ]),
                         'penalty': ['l1', 'l2', 'elasticnet']})

In [59]:
LR_grid_model.best_params_

{'C': 37.926901907322495, 'l1_ratio': 0.0, 'penalty': 'l1'}

In [61]:
cv_results = cross_validate(LogisticRegression(solver='saga', max_iter=5000, class_weight = "balanced", C=37.926901907322495, l1_ratio=0.0, penalty='l1'), X, y, cv=num_folds, scoring=
                                 scoring,  return_train_score=True)
results['LR_grid_model'] = {'fit_time': cv_results['fit_time'].mean(), 
                       'train_accuracy':  cv_results['train_accuracy'].mean(), 
                      'train_precision':  cv_results['train_precision'].mean(),
                      'train_recall':  cv_results['train_recall'].mean(),
                      'train_f1':  cv_results['train_f1'].mean(),
                      'score_time':   cv_results['score_time'].mean(), 
                      'test_accuracy':  cv_results['test_accuracy'].mean(), 
                      'test_precision':  cv_results['test_precision'].mean(),
                      'test_recall':  cv_results['test_recall'].mean(),
                      'test_f1':  cv_results['test_f1'].mean(),
                      }

##  b) Используйте RandomizedSearchCV.

## c) *Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.

## d) Повторите п. 5 после каждого итогового изменения параметров.

In [64]:
tuned_models=[
      
      {'name':'LR_tuned',"model": LogisticRegression(random_state=random_seed)  , 'params':{'C':[0.1,0.2,0.3,0.5,0.7,1], 'penalty':['l1', 'l2'], 'max_iter':[100,200,300,400,500]}},
      {'name':'DTC_tuned',"model": DecisionTreeClassifier(random_state=random_seed), 'params':{'criterion': ['gini', 'entropy', 'log_loss'], 'max_features': [None, 'log2', 'sqrt'], 
                                                                                                'max_depth': [2,3, 4,5, 6, 7, 8, 10, 20, 50]}},
      {'name':'RFC_tuned', "model": RandomForestClassifier(random_state=random_seed), 'params':{'criterion': ['gini', 'entropy', 'log_loss'], 'max_features': [None, 'log2', 'sqrt'], 
                                                                                                'max_depth': [2,3, 4,5, 6, 7, 8, 10, 20, 50]}},
      {'name':'BGC_tuned',"model": BaggingClassifier(random_state=random_seed), 'params':{'n_estimators':[10,25,50], 'max_samples':[3,5,7,9,11]}},
      
      {'name':'GBC_tuned',"model": GradientBoostingClassifier(random_state=random_seed), 'params':{'loss':['log_loss', 'deviance', 'exponential'], 
                                                                                          'learning_rate':[0.05, 0.1, 0.15,0.25], 'criterion': ['friedman_mse', 'squared_error']}},
      {'name':'ABC_tuned',"model": AdaBoostClassifier(random_state=random_seed), 'params':{'learning_rate':[0.05, 0.1, 0.15,0.25], 'n_estimators':[10,25,50]}}
   
]


In [68]:
res=[]
for v in  tuned_models:
    res.append((v['name'], RandomizedSearchCV(v['model'], v['params'], cv=10).fit(X_train, y_train)))
    cv_results = cross_validate(RandomizedSearchCV(v['model'], v['params'], cv=10), X, y, cv=num_folds, scoring=
                                 scoring,  return_train_score=True)
    results[v['name']] = {'fit_time': cv_results['fit_time'].mean(), 
                       'train_accuracy':  cv_results['train_accuracy'].mean(), 
                      'train_precision':  cv_results['train_precision'].mean(),
                      'train_recall':  cv_results['train_recall'].mean(),
                      'train_f1':  cv_results['train_f1'].mean(),
                      'score_time':   cv_results['score_time'].mean(), 
                      'test_accuracy':  cv_results['test_accuracy'].mean(), 
                      'test_precision':  cv_results['test_precision'].mean(),
                      'test_recall':  cv_results['test_recall'].mean(),
                      'test_f1':  cv_results['test_f1'].mean(),
                      }

# 7. Сформулируйте выводы по проделанной работе:

## a) Сравните метрики построенных моделей.

In [72]:
res1=pd.DataFrame.from_dict(results)

res1.head(20).style.format('{:.4f}', na_rep='-').background_gradient(cmap='Blues', axis=1)

,Base_LR,SS_DTS,SS_RFC,SS_BGS,SS_GBC,SS_ABC,SS_LGBMC,SS_XGBC,SS_SCC,SS_CBC,clf,LR_grid_model,LR_tuned,DTC_tuned,RFC_tuned,BGC_tuned,GBC_tuned,ABC_tuned
fit_time,0.0197,0.0141,0.2081,0.0425,0.1812,0.0972,0.0623,0.0961,1.0138,1.8073,0.0205,0.9231,2.3150,0.4558,17.8878,4.7387,13.5924,5.2349
train_accuracy,0.8690,1.0000,1.0000,0.9892,0.9521,0.8987,1.0000,1.0000,0.9999,0.9702,0.8690,0.8506,0.8723,0.8882,0.9614,0.8366,0.9338,0.8753
train_precision,0.8668,1.0000,1.0000,0.9936,0.9524,0.9070,1.0000,1.0000,0.9998,0.9758,0.8668,0.8692,0.8723,0.8919,0.9572,0.8471,0.9301,0.8730
train_recall,0.9020,1.0000,1.0000,0.9869,0.9615,0.9103,1.0000,1.0000,1.0000,0.9703,0.9020,0.8594,0.9011,0.9086,0.9746,0.8604,0.9521,0.9066
train_f1,0.8840,1.0000,1.0000,0.9902,0.9569,0.9086,1.0000,1.0000,0.9999,0.9730,0.8840,0.8643,0.8865,0.9000,0.9658,0.8535,0.9410,0.8895
score_time,0.0104,0.0089,0.0227,0.0105,0.0097,0.0154,0.0106,0.0109,0.0185,0.0101,0.0104,0.0058,0.0071,0.0046,0.0145,0.0110,0.0058,0.0134
test_accuracy,0.8484,0.7951,0.8430,0.8255,0.8549,0.8332,0.8550,0.8462,0.8386,0.8658,0.8484,0.8289,0.8516,0.8364,0.8462,0.8353,0.8528,0.8429
test_precision,0.8621,0.8305,0.8577,0.8680,0.8751,0.8527,0.8795,0.8731,0.8460,0.8803,0.8621,0.8637,0.8659,0.8549,0.8554,0.8545,0.8650,0.8572
test_recall,0.8676,0.7929,0.8617,0.8126,0.8635,0.8498,0.8578,0.8478,0.8716,0.8774,0.8676,0.8245,0.8715,0.8519,0.8735,0.8520,0.8754,0.8636
test_f1,0.8614,0.8074,0.8565,0.8350,0.8654,0.8473,0.8655,0.8556,0.8554,0.8764,0.8614,0.8391,0.8647,0.8501,0.8602,0.8494,0.8660,0.8558


## b) *Сравните с полученными результатами в домашнем задании по теме «Ансамблирование».

В начальных колоннках(1-11) датафрейма приведены метрики по моделям из ДЗ "Ансамблирование", в 12-17 - модели после оптимизации. Как можно заметить,
оптимизация во всех случаях привела к увеличению метрик качества моделей. Однако параллельно выросло и время обучения, в некоторых случаях - значительно (например, у RFC в 85 раз!, а метрики возросли не столь существенно).
При этом лучшие результаты на тестовой выборке продемонстрировал catBoostClassifier ,без всякого подбора гиперпараметров